In [75]:
%load_ext autoreload
%autoreload 2

import torch
from src.models.gps import GPS
from src.models.utils.hooks import GPSHook
from src.models.explainer.explainer_pipeline import ExplainerPipeline
from src.data import loader
from src.models.model import train, test

from src.models.gps import GPS
from src.models.gcn import GCN
from src.models.explainer.explainer_pipeline import ExplainerPipeline
from src.models.explainer.gnn_explainer import GNNExplainer
from src.models.explainer.attention_explainer import AttentionExplainer
from src.data import loader

import networkx as nx

from torch_geometric.explain import ModelConfig

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of src.models.explainer.attention_explainer failed: Traceback (most recent call last):
  File "/Users/ryanhung/anaconda3/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/Users/ryanhung/anaconda3/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/Users/ryanhung/anaconda3/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 936, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1074, in get_code
  File "<frozen importlib._bootstrap_external>", line 1004, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/Users/ryanhung/code/paying-attention/src/models/explainer/att

In [76]:
data, num_classes, data_networkx = loader.load_clean_bashapes(num_nodes=25, num_edges=5, num_motifs=10, laplacian_eigenvector_dimensions=2)

In [80]:
gps_params = {
    'pe_channels': 2,
    'num_layers': 4,
    'hidden_channels': 4,
    'num_attention_heads': 1,
    'observe_attention': True
}

gcn_params = {
    "hidden_channels": 20,
    "num_layers": 3
}

explainer_params = {
    'explanation_type': 'model',
    'node_mask_type': 'attributes',
    'edge_mask_type': 'object',
    'model_config': ModelConfig(
        mode='multiclass_classification',
        task_level='node',
        return_type='raw',
    )
}

explainer_pipeline = ExplainerPipeline(data, num_classes, GPS, explainer=AttentionExplainer, Hook=GPSHook, model_params=gps_params, explainer_params=explainer_params, epochs=2)
explainer_pipeline.get_accuracies()

100%|██████████| 2/2 [00:00<00:00, 14.82it/s]

Train accuracy: 0.3333333333333333
Test accuracy: 0.3333333333333333


In [85]:
gps = GPS(data, num_classes, **gps_params)
hook = GPSHook(gps)
train(gps, data, epochs=2)
hook.remove_hooks()
hook.attention_weights

100%|██████████| 2/2 [00:00<00:00, 14.66it/s]


{0: tensor([[[0.0133, 0.0133, 0.0133,  ..., 0.0129, 0.0129, 0.0129],
          [0.0133, 0.0133, 0.0133,  ..., 0.0129, 0.0129, 0.0129],
          [0.0133, 0.0133, 0.0133,  ..., 0.0129, 0.0129, 0.0129],
          ...,
          [0.0133, 0.0133, 0.0133,  ..., 0.0129, 0.0129, 0.0129],
          [0.0133, 0.0133, 0.0133,  ..., 0.0129, 0.0129, 0.0129],
          [0.0133, 0.0133, 0.0133,  ..., 0.0129, 0.0129, 0.0129]]]),
 1: tensor([[[0.0129, 0.0084, 0.0128,  ..., 0.0108, 0.0105, 0.0111],
          [0.0142, 0.0117, 0.0141,  ..., 0.0104, 0.0107, 0.0096],
          [0.0130, 0.0080, 0.0129,  ..., 0.0112, 0.0109, 0.0115],
          ...,
          [0.0126, 0.0074, 0.0131,  ..., 0.0178, 0.0176, 0.0186],
          [0.0131, 0.0069, 0.0135,  ..., 0.0174, 0.0170, 0.0181],
          [0.0116, 0.0080, 0.0121,  ..., 0.0184, 0.0183, 0.0191]]]),
 2: tensor([[[0.0135, 0.0142, 0.0136,  ..., 0.0135, 0.0135, 0.0136],
          [0.0105, 0.0132, 0.0111,  ..., 0.0095, 0.0088, 0.0105],
          [0.0130, 0.0104, 0.01

In [83]:
data.edge_index

tensor([[ 0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,
          4,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
          7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,
          9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 11,
         11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13,
         13, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16,
         16, 17, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19,
         20, 20, 20, 20, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 23, 23, 23, 23,
         23, 24, 24, 24, 24, 24, 25, 25, 25, 26, 26, 26, 27, 27, 28, 28, 29, 29,
         14, 26, 30, 30, 30, 31, 31, 31, 32, 32, 33, 33, 34, 34, 19, 32, 35, 35,
         35, 36, 36, 36, 37, 37, 38, 38, 39, 39, 13, 35, 40, 40, 40, 41, 41, 41,
         42, 42, 43, 43, 44,

In [84]:
explainer_pipeline.explain(26, laplacian_eigenvector_pe = data.laplacian_eigenvector_pe)
explainer_pipeline.explanations[26]

tensor([[0.0135, 0.0145, 0.0134,  ..., 0.0132, 0.0133, 0.0129],
        [0.0073, 0.0208, 0.0070,  ..., 0.0075, 0.0070, 0.0085],
        [0.0135, 0.0143, 0.0135,  ..., 0.0135, 0.0137, 0.0132],
        ...,
        [0.0138, 0.0176, 0.0138,  ..., 0.0159, 0.0168, 0.0149],
        [0.0138, 0.0176, 0.0138,  ..., 0.0154, 0.0163, 0.0145],
        [0.0136, 0.0183, 0.0137,  ..., 0.0165, 0.0175, 0.0156]])


AttributeError: 'dict' object has no attribute 'dim'

In [ ]:
def max_reduce(matrix):

    #apply max reduction along columns (dim = 1)
    max_values, _ = matrix.max(dim=1)
    return max_values

#finds the weighted_average for one singular attention matrix

#when summing across dim=0, output shows how much attention each node receives, when dim=1, it shows how much attention each node is giving

def weighted_average_received(attention_matrix):
    
    # Apply softmax across rows (dim=1) to normalize each row of the attention matrix
    softmax_attention = F.softmax(attention_matrix, dim=1)
    
    # Compute the weighted average across each row (dim=1) by summing
    weighted_avg = softmax_attention.sum(dim=0)  # Sum along the columns
    
    return weighted_avg

def weighted_average_given(attention_matrix):
    """Computes the weighted average of an attention matrix to show how much attention each node is giving to others"""
    
    # Step 1: Apply softmax across rows (dim=1) to normalize attention
    softmax_attention = F.softmax(attention_matrix, dim=1)
    
    # Step 2: Compute the weighted average across rows (dim=1)
    # Multiply each value by its respective column index (weighted sum)
    weighted_avg = torch.matmul(softmax_attention, torch.arange(attention_matrix.size(1), dtype=torch.float32))

    return weighted_avg

def weighted_average_all_layers(function, matrices):

    #store all the weighted averages per matrix (from each layer)
    weighted_averages = []

    #use the weighted_average function (single matrix use case) in a loop to collect all the weighted averages,
    #and append to list

    for matrix in matrices:
        current_weighted_avg = function(matrix)
        weighted_averages.append(current_weighted_avg)

    # Compute the average of the weighted averages across all layers
    avg_all_matrices= torch.stack(weighted_averages).mean(dim=0)

    return weighted_averages, avg_all_matrices

def top_k_nodes(matrix, top_k=0.1):

    # Rank nodes by importance (highest first)
    sorted_indices = torch.argsort(matrix, descending=True)

    # Select the top K nodes (either as a percentage or fixed number)
    if isinstance(top_k, float):
        top_k = int(len(sorted_indices) * top_k)  # Percentage to number of nodes
    top_nodes = sorted_indices[:top_k]
    return top_nodes